In [3]:
import boto3
from botocore import UNSIGNED
from botocore.client import Config
import os
import pandas as pd
from datetime import datetime

## Fuentes de datos


### Cliente de AWS dispuesto por IDEAM

- Para listar los archivos del 16 de septiembre de 2020 para el radar de Barrancabermeja se debe utilizar:

    `aws s3 --no-sign-request ls --recursive s3://s3-radaresideam/l2_data/2020/09/16`

    La bandera `--no-sign-request` le permitirá correr el comando sin la necesidad de proporcionar credenciales.

- Para descargar todos los datos del día 16 de septiembre de 2020 para el radar de Barrancabermeja se debe utilizar:

    `aws s3 --no-sign-request sync s3://s3-radaresideam/l2_data/2020/09/16/Barrancabermeja/ ./`

- Para descargar solo un archivo de radar en específico se debe utilizar:

    `aws s3 --no-sign-request cp s3://s3-radaresideam/l2_data/2020/09/16/Barrancabermeja/<NOMBRE DEL ARCHIVO> ./`

In [4]:
# Parámetros
bucket_name = 's3-radaresideam'
prefix = 'l2_data/2024/10/25/'  # Ruta del radar y fecha en el bucket

In [5]:
# Configuración del cliente S3 sin autenticación
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))

# Función para listar todos los archivos de una fecha específica en el bucket, junto con su fecha de última modificación
def listar_archivos(bucket, prefix):
    archivos = []
    continuation_token = None
    
    while True:
        if continuation_token:
            resultado = s3.list_objects_v2(Bucket=bucket, Prefix=prefix, ContinuationToken=continuation_token)
        else:
            resultado = s3.list_objects_v2(Bucket=bucket, Prefix=prefix)
        
        if 'Contents' in resultado:
            for archivo in resultado['Contents']:
                path_archivo = archivo['Key']
                ultima_modificacion = archivo['LastModified']
                
                # Extraer el radar del path (estructura: l2_data/año/mes/dia/radar/archivo)
                radar = path_archivo.split('/')[4]
                
                # Extraer el nombre del archivo
                nombre_archivo = path_archivo.split('/')[-1]
                
                # Verificar qué tipo de archivo estamos manejando para extraer fecha y hora
                if '-' in nombre_archivo:
                    # Estructura con fecha completa y hora (e.g., 9100SAN-20240911-220724-PPIVol.nc)
                    fecha_captura = nombre_archivo.split('-')[1]  # Extraer "20240911"
                    hora_captura = nombre_archivo.split('-')[2][:6]  # Extraer "220724"
                    
                    # Convertir fecha y hora en timestamp
                    try:
                        timestamp_captura = datetime.strptime(f'{fecha_captura} {hora_captura}', '%Y%m%d %H%M%S')
                    except ValueError:
                        timestamp_captura = None
                else:
                    # Estructura con fecha y hora combinada (e.g., TAB240911220835.RAW6L9F)
                    fecha_hora_captura = nombre_archivo[3:15]  # Extraer "240911220835"
                    fecha_captura = fecha_hora_captura[:6]  # Extraer "240911"
                    hora_captura = fecha_hora_captura[6:]  # Extraer "220835"
                    
                    # Convertir fecha y hora en timestamp (asumimos que el siglo es el 2000)
                    try:
                        timestamp_captura = datetime.strptime(f'20{fecha_captura} {hora_captura}', '%Y%m%d %H%M%S')
                    except ValueError:
                        timestamp_captura = None

                # Añadir la información a la lista
                archivos.append((path_archivo, radar, ultima_modificacion, timestamp_captura))
        
        # Verificamos si hay más archivos a listar (paginación)
        if resultado.get('IsTruncated'):
            continuation_token = resultado['NextContinuationToken']
        else:
            break
    
    return archivos


In [6]:
# Listar archivos con fechas de creación y radar
archivos = listar_archivos(bucket_name, prefix)

# Crear un DataFrame a partir de la lista de archivos
df = pd.DataFrame(archivos, columns=['path_archivo', 'radar', 'ultima_modificacion', 'timestamp_captura'])

In [7]:
df

,path_archivo,radar,ultima_modificacion,timestamp_captura
0,l2_data/2024/10/25/Bogota/1399BOG-20241025-000...,Bogota,2024-10-25 00:06:22+00:00,2024-10-25 00:00:36
1,l2_data/2024/10/25/Bogota/1399BOG-20241025-000...,Bogota,2024-10-25 00:10:40+00:00,2024-10-25 00:00:36
2,l2_data/2024/10/25/Bogota/1399BOG-20241025-000...,Bogota,2024-10-25 00:10:43+00:00,2024-10-25 00:01:38
3,l2_data/2024/10/25/Bogota/1399BOG-20241025-000...,Bogota,2024-10-25 00:10:41+00:00,2024-10-25 00:02:40
4,l2_data/2024/10/25/Bogota/1399BOG-20241025-000...,Bogota,2024-10-25 00:15:14+00:00,2024-10-25 00:02:40
...,...,...,...,...
16502,l2_data/2024/10/25/santa_elena/9100SAN-2024102...,santa_elena,2024-10-25 23:58:53+00:00,2024-10-25 23:48:05
16503,l2_data/2024/10/25/santa_elena/9100SAN-2024102...,santa_elena,2024-10-25 23:58:53+00:00,2024-10-25 23:49:06
16504,l2_data/2024/10/25/santa_elena/9100SAN-2024102...,santa_elena,2024-10-25 23:58:53+00:00,2024-10-25 23:50:08
16505,l2_data/2024/10/25/santa_elena/9100SAN-2024102...,santa_elena,2024-10-25 23:58:53+00:00,2024-10-25 23:51:22


## Análisis de frecuencia en la disponibilidad de los datos

### Captura de los datos 
Cada cuanto se capturan los datos

In [8]:
# Ordenar los datos por radar y por timestamp de captura
df = df.sort_values(by=['radar', 'timestamp_captura'])

# Calcular la diferencia entre los timestamps de captura por radar en minutos
df['diferencia_captura_minutos'] = df.groupby('radar')['timestamp_captura'].diff().dt.total_seconds() / 60

# Eliminar las diferencias de captura que sean 0 minutos
df_no_ceros_captura = df[df['diferencia_captura_minutos'] != 0]

# Crear un resumen para cada radar mostrando las diferencias entre las capturas de datos (media, mínima y máxima)
resumen_captura_radar = df_no_ceros_captura.groupby('radar')['diferencia_captura_minutos'].agg(['mean', 'min', 'max', 'count']).reset_index()

In [9]:
resumen_captura_radar

,radar,mean,min,max,count
0,Bogota,1.251037,1.016667,7.200000,1141
1,Carimagua,0.545222,0.200000,106.450000,2630
2,Corozal,0.503470,0.416667,1.516667,2858
3,Guaviare,0.982269,0.566667,2.600000,1456
4,Munchique,3.690451,0.600000,743.483333,384
5,San_Andres,0.519741,0.416667,11.066667,2759
6,Tablazo,0.559543,0.416667,1.550000,2562
7,santa_elena,1.304888,0.533333,2.216667,1098


### Guardado de los datos
Cada cuanto se guarda en el repositorio los datos

In [10]:
# Ordenar los datos por radar y por timestamp de guardado
df = df.sort_values(by=['radar', 'ultima_modificacion'])

# Calcular la diferencia entre los timestamps de guardado por radar en minutos
df['diferencia_guardado_minutos'] = df.groupby('radar')['ultima_modificacion'].diff().dt.total_seconds() / 60

# Eliminar las filas donde la diferencia de guardado es 0
df_no_ceros = df[df['diferencia_guardado_minutos'] != 0]

# Crear un resumen para cada radar mostrando las diferencias entre las cargas (media, mínima y máxima)
resumen_guardado_radar = df_no_ceros.groupby('radar')['diferencia_guardado_minutos'].agg(['mean', 'min', 'max', 'count']).reset_index()

In [11]:
resumen_guardado_radar

,radar,mean,min,max,count
0,Bogota,1.571264,0.016667,10.733333,910
1,Carimagua,3.528465,0.016667,119.966667,404
2,Corozal,2.786919,0.016667,9.966667,516
3,Guaviare,3.647066,0.016667,9.100000,392
4,Munchique,8.135441,0.016667,744.950000,174
5,San_Andres,3.157043,0.016667,18.550000,452
6,Tablazo,3.251553,0.016667,14.433333,440
7,santa_elena,3.378263,0.016667,10.883333,424


### Extensiones de los archivos

Cuales son las extensiones de los archivos en el repositorio

In [12]:
def obtener_extension(path):
    nombre_archivo = path.split('/')[-1]
    if '.' in nombre_archivo:
        return nombre_archivo.split('.')[-1]
    else:
        # Si no tiene extensión, tomar los últimos caracteres que identifican el tipo de archivo
        return nombre_archivo[-7:]  # Consideramos una longitud razonable para la extensión

In [13]:
# Aplicamos la función para obtener las extensiones correctas
df['extension'] = df['path_archivo'].apply(obtener_extension)

# Contamos las extensiones nuevamente
conteo_extensiones = df['extension'].value_counts().reset_index()

# Renombrar columnas para claridad
conteo_extensiones.columns = ['extension', 'count']


In [14]:
conteo_extensiones

,extension,count
0,nc,2241
1,gz,761
2,RAWXVUS,1
3,RAW22LN,1
4,RAWXPBP,1
...,...,...
13502,RAW24X6,1
13503,RAW24X8,1
13504,RAW24XC,1
13505,RAWXRTM,1


In [15]:
def agrupar_extension(extension):
    if 'RAW' in extension:
        return 'RAW'
    return extension

In [16]:
# Aplicamos la función de agrupación sobre la columna de extensión
df['extension_agrupada'] = df['extension'].apply(agrupar_extension)

# Contamos las extensiones nuevamente después de la agrupación
conteo_extensiones_agrupado = df['extension_agrupada'].value_counts().reset_index()

# Renombrar las columnas para mayor claridad
conteo_extensiones_agrupado.columns = ['extension', 'count']

In [17]:
conteo_extensiones_agrupado

,extension,count
0,RAW,13505
1,nc,2241
2,gz,761


In [18]:
# Extensiones por radar
df[['radar', 'extension_agrupada']].groupby(['radar', 'extension_agrupada']).size().reset_index(name='conteo')

,radar,extension_agrupada,conteo
0,Bogota,gz,761
1,Bogota,nc,1142
2,Carimagua,RAW,2631
3,Corozal,RAW,3146
4,Guaviare,RAW,1457
5,Munchique,RAW,385
6,San_Andres,RAW,3036
7,Tablazo,RAW,2850
8,santa_elena,nc,1099


## Descargar datos

In [19]:
# Función para descargar un archivo específico
def descargar_archivo(bucket, file_key, destino_local):
    try:
        s3.download_file(bucket, file_key, destino_local)
        print(f'Archivo {file_key} descargado exitosamente a {destino_local}.')
    except Exception as e:
        print(f'Error descargando el archivo {file_key}: {e}')

In [20]:
# Archivo objetivo
file_origin = 'l2_data/2024/10/25/Carimagua/CAR241025181428.RAWXTKP'
file_destination = 'CAR241025181428.RAWXTKP'

In [21]:
descargar_archivo(bucket_name, file_origin, file_destination)

Archivo l2_data/2024/10/25/Carimagua/CAR241025181428.RAWXTKP descargado exitosamente a CAR241025181428.RAWXTKP.
